# Digging Into the Pronto Data Release

In October, [Pronto CycleShare](https://www.prontocycleshare.com/), turned one year old and released a treasure-trove of data on the 140,000 individual rides trips during the first year.
Here I want to dig into this data and answer a few questions:

- many naysayers insist that Seattle is too cold, too wet, too hilly to be a world class bicycling city. How do these elements actually affect users of the Pronto system?
- what is the difference in usage by annual members and short-term users? How might Pronto evolve to be more useful to these groups?
- how do Pronto trips compare to trips by other cyclists in the city? Can characteristics of Pronto use give us insight into deeper trends within the city?
- Can we cleverly de-anonymize the data and learn about the usage patterns of individual members?

If you're interested in how the plots and figures below were created, I have made available all the Python code I used to run this analysis. For details, see http://github.com/jakevdp/ProntoData.